In [8]:
import textwrap
import chromadb
import numpy as np
import pandas as pd
import re

from chromadb import Documents, EmbeddingFunction, Embeddings

from google import genai

client = genai.Client(api_key='')


for m in client.models.list():
  if 'embedContent' in m.supported_actions:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [2]:
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    EMBEDDING_MODEL_ID = "models/embedding-001"
    title = "Custom query"
    response = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=input,
        config=types.EmbedContentConfig(
          task_type="retrieval_document",
          title=title
        )
    )
    # 모든 문서의 임베딩 벡터를 리스트로 반환
    return [e.values for e in response.embeddings]

In [3]:
import json
#chroma_client.delete_collection("my_collection")
def preprocess_metadata(metadata):
    new_metadata = {}
    for k, v in metadata.items():
        if isinstance(v, list):
            new_metadata[k] = ", ".join(map(str, v))  # 리스트를 문자열로 변환
        else:
            new_metadata[k] = v
    return new_metadata
def batch_add(collection, documents, metadatas, ids, batch_size=100):
    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i+batch_size]
        batch_metas = metadatas[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]
        collection.add(
            documents=batch_docs,
            metadatas=batch_metas,
            ids=batch_ids
        )
def create_chroma_db(json_data, name):
    import chromadb

    chroma_client = chromadb.Client()
    db = chroma_client.create_collection(
        name=name,
        embedding_function=GeminiEmbeddingFunction()
    )

    # JSON 데이터에서 텍스트, 메타데이터, id 추출
    documents = [item["text"] for item in json_data]
    metadatas = [preprocess_metadata(item["metadata"]) for item in json_data]
    ids = [str(i) for i in range(len(json_data))]
    
    batch_add(db, documents, metadatas, ids, batch_size=100)
   
    return db
with open('disease_rag_with_metadata.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
db = create_chroma_db(data, "my_collection")

C:\Users\eh589\AppData\Local\Temp\ipykernel_3112\2476913475.py:27: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GeminiEmbeddingFunction()


In [4]:
sample_data = db.get(include=['documents', 'embeddings'])

df = pd.DataFrame({
    "IDs": sample_data['ids'][:3],
    "Documents": sample_data['documents'][:3],
    "Embeddings": [str(emb)[:50] + "..." for emb in sample_data['embeddings'][:3]]  # Truncate embeddings
})

print(df)

  IDs                                          Documents  \
0   0  증상: Fever, Fatigue, Difficulty Breathing이(가) 있...   
1   1  증상: Cough, Fatigue이(가) 있고, 나이: 25세, 성별: Female...   
2   2  증상: Cough, Fatigue이(가) 있고, 나이: 25세, 성별: Female...   

                                          Embeddings  
0  [ 2.24745572e-02 -7.10671246e-02 -3.98193412e-...  
1  [ 6.27511144e-02 -6.39859959e-02 -7.34391063e-...  
2  [ 5.98195456e-02 -6.23048060e-02 -7.82496259e-...  


In [37]:
def get_relevant_passage(query, db, n_results=5):
  results = db.query(query_texts=[query], n_results=n_results, include=['documents', 'metadatas'])
  passages = []
  for i in range(len(results['documents'][0])):
      doc = results['documents'][0][i]
      meta = results['metadatas'][0][i]
      
      passage_text = f"{doc} (나이: {meta.get('age', '정보 없음')}, 성별: {meta.get('gender', '정보 없음')}, 혈압: {meta.get('blood_pressure', '정보 없음')}, 콜레스테롤: {meta.get('cholesterol', '정보 없음')})"
      passages.append(passage_text)
  return passages
# Perform embedding search
passages = get_relevant_passage("Fever, Cough, Difficulty Breathing", db, 5)

# LLM 응답 자체 평가 함수 정의
def evaluate_response(user_query: str, llm_response: str, relevant_passages: list) -> dict: #
    """
    LLM의 답변을 자체적으로 평가하는 함수.
    주어진 프롬프트 지시사항을 얼마나 잘 따랐는지에 초점을 맞춥니다.
    """
    evaluation_results = {
        "score": 0,
        "criteria": {
            "overall_length_sufficient": False,
            "includes_greeting_and_symptom": False,
            "mentions_patient_profile_integrated": False,
            "mentions_disease_and_type": False,
            "includes_general_advice_list": False,
            "avoids_extreme_disease_direct_recommendation": True, # 변경된 로직으로 평가
            "avoids_direct_medical_advice": True,
            "uses_korean_for_english_terms": True,
            "matches_query_symptoms": False,
            "includes_closing_remark": False
        },
        "feedback": []
    }

    llm_response_lower = llm_response.lower()

    # 0. 답변 길이 충분성 확인
    word_count = len(llm_response.split())
    if word_count >= 100:
        evaluation_results["criteria"]["overall_length_sufficient"] = True
        evaluation_results["score"] += 2
    else:
        evaluation_results["feedback"].append(f"피드백: 답변 길이가 {word_count} 단어로 충분하지 않습니다 (최소 100단어 필요).")

    # 1. 시작 인사 및 증상 언급 확인
    if re.search(r'^(안녕하세요|환영합니다).*(기침|콧물|열|피로|불편)', llm_response, re.IGNORECASE | re.DOTALL):
        evaluation_results["criteria"]["includes_greeting_and_symptom"] = True
        evaluation_results["score"] += 1
    else:
        evaluation_results["feedback"].append("피드백: 답변이 시작 인사와 사용자 증상 언급으로 시작하지 않습니다.")

    # 2. 환자 프로필 정보 통합 여부 확인
    profile_keywords = ['나이:', '세', '성별:', '혈압:', '콜레스테롤:']
    if any(keyword in llm_response for keyword in profile_keywords) or \
       re.search(r'(\d+세\s*(남성|여성)|(Low|Normal|High)\s*혈압|Low|Normal|High)\s*콜레스테롤', llm_response, re.IGNORECASE):
        evaluation_results["criteria"]["mentions_patient_profile_integrated"] = True
        evaluation_results["score"] += 3
    else:
        evaluation_results["feedback"].append("피드백: 답변에 환자 프로필 정보(나이, 성별, 혈압, 콜레스테롤)가 자연스럽게 통합되지 않았습니다.")

    # 3. 질병 언급 및 일반적/심각성 구분
    mentioned_diseases = re.findall(r'\b([가-힣A-Za-z\s]+)\((Common Cold|Influenza|Asthma|Eczema|Migraine|Malaria)\)', llm_response) #
    if mentioned_diseases:
        evaluation_results["criteria"]["mentions_disease_and_type"] = True
        evaluation_results["score"] += 1
    else:
        evaluation_results["feedback"].append("피드백: 답변에 질병이 명확히 언급되지 않았거나, 일반적/심각성 구분이 부족합니다.")

    # 4. 일반적인 조언 목록 포함 여부
    if re.search(r'-\s*\*\*.+\*\*:', llm_response):
        evaluation_results["criteria"]["includes_general_advice_list"] = True
        evaluation_results["score"] += 2
    else:
        evaluation_results["feedback"].append("피드백: 답변에 일반적인 조언이 목록 형태로 충분히 포함되지 않았습니다.")

    # 5. 극단적이거나 심각한 질병 직접 추천 회피 여부 (로직 변경)
    extreme_diseases = ["Cancer", "Stroke", "Malaria", "Ebola Virus", "HIV/AIDS", "Tuberculosis", "Alzheimer's Disease", "Liver Cancer", "Kidney Cancer", "Pancreatic Cancer"] #
    mitigating_phrases = [
        r'드물게는', r'만약', r'다른', r'일\s*수도\s*있습니다', r'가능성\s*이\s*있습니다', r'관련이\s*있을\s*수\s*있습니다', r'고려해\s*볼\s*수\s*있습니다'
    ]
    
    for disease in extreme_diseases:
        # 질병 이름이 답변에 직접적으로 언급되었는지 확인
        disease_pattern = r'\b' + re.escape(disease) + r'\b'
        
        # 완화 표현 없이 질병이 바로 언급된 경우를 탐지
        # (완화 표현 뒤에 나오는 질병은 OK)
        if re.search(disease_pattern, llm_response, re.IGNORECASE):
            is_mitigated = False
            # 질병 주변 텍스트에서 완화 표현을 찾습니다.
            # 예: "드물게는 말라리아일 수도 있습니다."
            # Look for mitigating phrases *before* the disease in the sentence/phrase.
            # This is a heuristic and might not catch all nuances, but avoids PatternError.
            
            # Find all matches of the disease
            for match in re.finditer(disease_pattern, llm_response, re.IGNORECASE):
                start_index = match.start()
                # Check a window before the disease mention
                context_before = llm_response[max(0, start_index - 50):start_index] # 50자 앞까지 확인
                
                for mp in mitigating_phrases:
                    if re.search(mp, context_before, re.IGNORECASE):
                        is_mitigated = True
                        break
                if is_mitigated:
                    break # 이 질병 언급은 완화되었으니 다음 질병으로 넘어감

            if not is_mitigated:
                evaluation_results["criteria"]["avoids_extreme_disease_direct_recommendation"] = False
                evaluation_results["score"] -= 5
                evaluation_results["feedback"].append(f"주의: 답변에 '{disease}'와 같은 심각한 질병이 직접적으로 언급되었습니다 (완화 표현 없음).")
                # break # 한 번이라도 직접 언급되면 바로 감점 (선택)
                # 만약 모든 극단적 질병 언급을 확인하려면 이 break를 주석 처리
                # 현재는 첫 직접 언급에서 감점 후 다음 극단적 질병으로 넘어가는 방식

    # 6. 직접적인 의료 조언 회피 여부 (동일)
    forbidden_phrases = ["병원 방문", "전문가 상담", "의사와 상담", "진찰이 필요", "치료를 받아야", "약물 복용", "정확한 진단"]
    for phrase in forbidden_phrases:
        if phrase in llm_response:
            evaluation_results["criteria"]["avoids_direct_medical_advice"] = False
            evaluation_results["score"] -= 10
            evaluation_results["feedback"].append(f"심각: 금지된 의료 권고 문구 '{phrase}'가 포함되었습니다.")
            break

    # 7. 영어 단어 한글 뜻 제공 여부 (동일)
    english_terms_in_passage = re.findall(r'[A-Za-z]+(?=\()', " ".join(relevant_passages)) #
    for term in set(english_terms_in_passage):
        if not re.search(fr'\b{re.escape(term)}\b\([가-힣]+\)', llm_response):
            evaluation_results["criteria"]["uses_korean_for_english_terms"] = False
            evaluation_results["score"] -= 0.5
            evaluation_results["feedback"].append(f"피드백: 영어 단어 '{term}'에 대한 한글 뜻이 누락되었을 수 있습니다.")

    # 8. 쿼리 증상 매칭 확인 (동일)
    query_symptoms_list = []
    if "기침" in user_query: query_symptoms_list.append("기침")
    if "콧물" in user_query: query_symptoms_list.append("콧물")
    if "열" in user_query: query_symptoms_list.append("열")
    if "피로" in user_query: query_symptoms_list.append("피로")

    all_query_symptoms_mentioned = True
    for symptom in query_symptoms_list:
        if symptom not in llm_response:
            all_query_symptoms_mentioned = False
            evaluation_results["feedback"].append(f"피드백: 사용자 쿼리의 핵심 증상 '{symptom}'이 답변에 언급되지 않았습니다.")
            break

    if all_query_symptoms_mentioned:
        evaluation_results["criteria"]["matches_query_symptoms"] = True
        evaluation_results["score"] += 1

    # 9. 마무리 인사 포함 여부
    if re.search(r'(더 궁금한 점이 있으시면 언제든지 다시 질문해주세요\.|항상 건강하시길 바랍니다\.|궁금한 점이 있다면 언제든지 문의해주세요\.)', llm_response, re.DOTALL):
        evaluation_results["criteria"]["includes_closing_remark"] = True
        evaluation_results["score"] += 1
    else:
        evaluation_results["feedback"].append("피드백: 답변에 마무리 인사가 포함되지 않았습니다.")

    return evaluation_results

In [39]:
def make_prompt(query, relevant_passages):
  escaped = " ".join([p.replace("'", "").replace('"', "").replace("\n", " ") for p in relevant_passages])
  prompt = f"""
  당신은 사용자의 증상과 개인 프로필 정보를 기반으로 질병을 설명하고, **일상생활에서 할 수 있는 구체적이고 실용적인 조언을 상세하게 제공하는** 의료 상담 도우미입니다. 당신의 답변은 정보가 풍부하고 친절하며, **극단적이거나 심각한 질병을 직접적으로 진단하거나 추천하는 뉘앙스를 피해야 합니다.** 답변은 최소 100단어 이상으로 작성해 주세요.

  **단계별 지시사항:**
  1. 사용자 질문을 이해하고 핵심 증상(예: 기침, 콧물)을 정확하고 상세하게 파악하세요.
  2. 제공된 '관련 정보 (PASSAGE)'를 면밀히 검토하여 사용자 증상과 가장 밀접하게 일치하는 질병(들)을 식별하되, **데이터에 기반한 질병 연관성을 언급하되 불필요하게 심각성을 강조하지 마세요.**
  3. **여러 질병이 검색될 경우, 가장 일반적이거나 흔한 질환(예: 감기, 알레르기)을 우선적으로 상세히 설명하고, 그 다음으로 관련된 다른 질병들도 간략하게 제시하세요.**
  4. 나이, 성별, 혈압, 콜레스테롤 수치와 같은 환자 프로필 정보가 있다면, 이를 **답변의 서론 부분에 해당 질병이 특정 프로필의 환자에게서 관찰될 수 있는 '사례'로 자연스럽게 통합하여 설명의 깊이를 더하세요.** 질병 진단의 직접적인 근거로 오해되지 않도록 주의하세요.
  5. 답변은 정보가 풍부하고 명확하며 친절하게 작성하며, 다음 **상세 권장 출력 형식**을 따르되, **세부적인 구문은 모델의 자연스러운 생성에 맡기세요.**

  **상세 권장 출력 형식:**
  안녕하세요! [사용자 질문에서 파악된 증상]이(가) 있으시군요. 불편하시겠지만, 몇 가지 가능한 원인과 생활 속 대처법을 함께 알아보겠습니다.

  (선택적: 임상 데이터에 따르면, [관련 정보의 나이]세 [관련 정보의 성별] 환자 중 [관련 정보의 혈압] 혈압과 [관련 정보의 콜레스테롤] 콜레스테롤 수치를 가진 분들에게서 [해당 질병과 연결된 증상]이 관찰된 사례가 있습니다.) 이러한 증상들은 [관련 정보에서 찾은 가장 일반적이고 가능성 높은 질병]과 관련이 있을 수 있습니다.

  [질병에 대한 간략한 추가 설명 (2-3문장)]. 이 질병의 일반적인 경과나 특징에 대해 간략히 설명해 주세요.

  이럴 때는 다음과 같은 생활 습관 개선을 통해 증상 완화에 도움을 줄 수 있습니다:
  - **충분한 휴식:** 몸이 회복하는 데 필요한 시간을 주세요.
  - **수분 섭취:** 따뜻한 물, 차 등을 자주 마셔 목을 촉촉하게 유지하고 탈수를 예방하세요.
  - **실내 환경 관리:** 적절한 실내 습도를 유지하고 환기를 자주 해주세요.
  - **영양가 있는 음식 섭취:** 면역력 강화를 위해 비타민과 미네랄이 풍부한 음식을 드세요.
  - [추가적인 일반적인 조언 1 (예: 스트레스 관리, 가벼운 운동 등)]
  - [추가적인 일반적인 조언 2 (예: 마스크 착용, 손 씻기 등)]

  만약 [사용자 질문에서 파악된 증상] 외에 다른 불편한 증상이 있거나, 현재 증상이 나아지지 않고 오히려 심해진다면 [다른 관련 질병]일 수도 있습니다. (이때, 극단적인 질병은 가급적 언급하지 않거나, "드물게는 ~일 수도 있습니다"와 같이 조심스러운 표현을 사용하세요.)

  더 궁금한 점이 있으시면 언제든지 다시 질문해주세요. 항상 건강하시길 바랍니다.

  아래는 참고할 수 있는 임상 데이터입니다:
  - 사용자 질문 (QUESTION): \"{query}\"
  - 관련 정보 (PASSAGE): \"{escaped}\"

  **주의사항:**
  - 병원 방문 및 전문적인 상담을 직접적으로 권유하는 문구는 최종 답변에 포함하지 마세요.
  - PASSAGE에 영어 단어가 포함되어 있다면, 괄호 안에 한글 뜻을 함께 제공해 주세요.
  - **제공된 정보 내에서 '기침'과 '습진'의 연관성이 있더라도, '기침'이라는 증상에 더 일반적이고 흔한 질병(예: Common Cold, Influenza)이 있다면 이를 우선적으로 고려하여 답변하세요.**
  - **'말라리아'와 같이 심각한 질병은 사용자가 직접적으로 언급하지 않는 한, 일반적인 증상만으로는 추천하지 마세요.**

  ANSWER:
  """.format(query=query, relevant_passages=escaped)
  return prompt
# 예시 사용
query = "나이: 25세, 성별: Male, 혈압: Normal, 콜레스테롤: Normal, 증상 : 심각한 공부하기 싫음"
passages = get_relevant_passage(query, db, 5)
prompt = make_prompt(query, passages)

#print(prompt)

MODEL_ID = "gemini-2.0-flash"
answer = client.models.generate_content(
    model = MODEL_ID,
    contents = prompt
)
# 변경 시작: 'ANSWER:' 이후의 텍스트만 추출하고 앞뒤 공백 제거
final_answer = answer.text.split("ANSWER:")
if len(final_answer) > 1:
    final_answer = final_answer[1].strip()
else:
    final_answer = answer.text.strip()
   
#LLM 답변 자체 평가
evaluation_results = evaluate_response(query, final_answer, passages)
#결과 출력
# 6. 결과 출력
print("--- LLM 답변 ---")
print(final_answer)
print("\n--- 자체 평가 결과 ---")
print(f"총점: {evaluation_results['score']}점")
print("기준별 평가:")
for criterion, value in evaluation_results['criteria'].items():
    print(f"  - {criterion}: {value}")
if evaluation_results['feedback']:
    print("피드백:")
    for fb in evaluation_results['feedback']:
        print(f"  - {fb}")

--- LLM 답변 ---
안녕하세요! 심각한 공부하기 싫음 증상이 있으시군요. 학업에 어려움을 느끼시는 것 같아 안타깝습니다. 몇 가지 가능한 원인과 생활 속 대처법을 함께 알아보겠습니다.

임상 데이터에 따르면, 25세 남성 환자 중 정상 혈압과 정상 콜레스테롤 수치를 가진 분들에게서 무기력감이나 집중력 저하와 같은 증상이 관찰된 사례가 있습니다. 이러한 증상들은 번아웃 증후군과 관련이 있을 수 있습니다.

번아웃 증후군은 장기간에 걸쳐 과도한 스트레스에 시달려 신체적, 정신적으로 극도의 피로감을 느끼는 상태를 말합니다. 단순히 공부가 싫은 감정을 넘어, 무기력감, 집중력 저하, 짜증, 수면 장애 등 다양한 증상을 동반할 수 있습니다.

이럴 때는 다음과 같은 생활 습관 개선을 통해 증상 완화에 도움을 줄 수 있습니다:
- **충분한 휴식:** 잠시 학업에서 벗어나 좋아하는 활동을 하며 휴식을 취하세요. 주말이나 휴가를 이용하여 여행을 가거나, 취미 생활을 즐기는 것도 좋은 방법입니다.
- **규칙적인 생활 습관:** 규칙적인 수면 시간을 지키고, 균형 잡힌 식사를 하세요.
- **운동:** 가벼운 운동은 스트레스 해소와 활력 증진에 도움이 됩니다. 산책, 조깅, 요가 등 자신에게 맞는 운동을 꾸준히 해보세요.
- **스트레스 관리:** 스트레스 해소를 위한 자신만의 방법을 찾으세요. 친구와 수다를 떨거나, 영화를 보거나, 음악을 듣는 등 좋아하는 활동을 통해 스트레스를 해소할 수 있습니다.
- **긍정적인 생각:** 긍정적인 마음가짐을 유지하도록 노력하세요. 자신을 격려하고 칭찬하며, 작은 성취에도 만족감을 느끼도록 노력하세요.

만약 심각한 공부하기 싫음 외에 다른 불편한 증상(예: 심한 불안, 우울감)이 있거나, 현재 증상이 나아지지 않고 오히려 심해진다면 다른 심리적인 어려움일 수도 있습니다. 이럴 때는 전문가의 도움을 받는 것이 좋습니다.

더 궁금한 점이 있으시면 언제든지 다시 질문해주세요. 항상 건강하시길 바랍니다.

--- 자체 평가 결과 -